In [1]:
!pip install SpeechRecognition


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 56.1 MB/s eta 0:00:00


In [1]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '1'
os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'
os.environ['DNNL_ENGINE_LIMIT_CPU_CAPABILITIES'] = '0'
os.environ['DNNL_VERBOSE'] ='1'
os.environ['SYCL_DEVICE_FILTER'] = 'opencl:gpu'

In [2]:
!pip install google-cloud-texttospeech

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 3.6 MB/s eta 0:00:00


In [3]:
!sudo apt-get install portaudio19-dev


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 16 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 1s (231 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEn

In [4]:
!pip install google-cloud-texttospeech google-auth speechrecognition


In [16]:
import os
import json
from google.cloud import texttospeech_v1
import random
import speech_recognition as sr

# Set the path to your JSON key file
json_key_path = "/content/prefab-wave-395613-26155198bb68.json"

# Set up Google Cloud Text-to-Speech client using the service account key
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = json_key_path
client = texttospeech_v1.TextToSpeechClient()

# Load intents from intents.json
with open('/content/intents.json') as json_data:
    intents = json.load(json_data)['intents']

# Initialize the recognizer
recognizer = sr.Recognizer()

# Function to convert text to speech and save as an MP3 file
def text_to_speech(text, output_file):
    synthesis_input = texttospeech_v1.SynthesisInput(text=text)
    voice = texttospeech_v1.VoiceSelectionParams(
        language_code="en-US", ssml_gender=texttospeech_v1.SsmlVoiceGender.NEUTRAL
    )
    audio_config = texttospeech_v1.AudioConfig(
        audio_encoding=texttospeech_v1.AudioEncoding.LINEAR16
    )
    response = client.synthesize_speech(
        input=synthesis_input, voice=voice, audio_config=audio_config
    )

    # Save the audio to a WAV file
    with open(output_file, "wb") as out:
        out.write(response.audio_content)

# Collect the WAV file path from the user
wav_file_path = input("Enter the path to the input WAV file: ")
output_mp3_file_path = input("Enter the path to save the output MP3 file: ")

def process_input(input_text):
    input_text = input_text.lower()
    response = "I'm sorry, I don't have information on that."

    for intent in intents:
        tag = intent["tag"]
        patterns = intent["patterns"]
        responses = intent["responses"]

        for pattern in patterns:
            if pattern.lower() in input_text or any(word in input_text for word in pattern.lower().split()):
                response = random.choice(responses)
                break

    return response

# Process the WAV file
with sr.AudioFile(wav_file_path) as source:
    print("Processing the WAV file...")
    audio = recognizer.record(source)
    try:
        input_text = recognizer.recognize_google(audio)
        print("You said:", input_text)

        response = process_input(input_text)
        print("Response:", response)

        # Convert the response text to speech and save as a WAV file named "response.wav"
        response_wav_file_path = os.path.join(output_mp3_file_path, "response.wav")
        text_to_speech(response, response_wav_file_path)
        print("Output WAV saved at:", response_wav_file_path)

    except sr.UnknownValueError:
        print("Sorry, could not understand audio")
    except sr.RequestError as e:
        print("Could not request results; {0}".format(e))


Enter the path to the input WAV file: /content/voice.wav
Enter the path to save the output MP3 file: /content/response
Processing the WAV file...
You said: abdomen pain
Response: 1)Stop any bleeding. Apply pressure to the wound with a sterile bandage, a clean cloth or a clean piece of clothing. 2)Immobilize the injured area. Don't try to realign the bone or push a bone that's sticking out back in. If you've been trained in how to splint and professional help isn't readily available, apply a splint to the area above and below the fracture sites. Padding the splints can help reduce discomfort. 3)Apply ice packs to limit swelling and help relieve pain. Don't apply ice directly to the skin. Wrap the ice in a towel, piece of cloth or some other material. 4)Treat for shock. If the person feels faint or is breathing in short, rapid breaths, lay the person down with the head slightly lower than the trunk and, if possible, elevate the legs.
Output WAV saved at: /content/response/response.wav
